In [ ]:
import Node

### Agent
Fields:
`Agent(inputs, behavioral_model, system_estimate, reference, reference_update, sensor_function, comparator_function, controller_update, internal_model_update)`
<br/>
**inputs** - source of inputs (i.e. environment, state observations, others behaviors)
<br/>
**behavioral model** - initial matrix expressing future state given last behavior ("B" matrix)
<br/>
**system estimate** - initial matrix expressing future state given last state ("A" matrix)
<br/>
**reference** - target state 
<br/>
**reference update** - update to reference (target) signal
<br/>
**sensor function** - for taking in (and processing) observations
<br/>
**comparator function** - for comparing observation and estimate
<br/>
**controller** - for generating output
<br/>
**controller update** - for updating the behavioral matrix 
<br/>
**internal model** - for generating forward prediction
<br/>
**internal model update** - for updating the system estimate 
<br/>


In [1]:
import numpy as np

In [2]:
# behavioral matrix (maps previous behavior to future behavior commands - how should I behave?)
B1 = np.array([[0.4,0,0],
               [0,0.3,0],
               [0,0,0.2]])
B2 = np.array([[0.1,0,0],
               [0,0.7,0],
               [0,0,0.5]])
# state/system matrix (maps previous state to future state - how will others behave (give past behaviors)?)
A1 = np.array([[0.3,0,0],
               [0,0.3,0],
               [0,0,0.3]])
A2 = np.array([[0.3,0,0],
               [0,0.3,0],
               [0,0,0.3]])
# reference matrix (how do I expect/want others to behave? target values)
R1 = np.array([0.3,0.5,0.1])
R2 = np.array([0.1,0.4,0.8])

# get the state observation (behaviors of others)
def sense(inputs):
    return inputs

# contrast reference with observations (and potentially incorporate predictions from internal model)
def compare(inputs, reference, prediction):
    error = reference - inputs
    return error

# generate a behavior
def control(error, behavioral_model, previous_behavior):
    #learning_rate = 0.01
    output = behavioral_model * previous_behavior
    return output


# adjusts behavioral model to get closer to behavior given 
def control_update(error, behavioral_model, previous_behavior, learning_rate):
    # LMS algorithm
    #learning_rate = 0.01
    behavioral_model = behavioral_model + learning_rate * error * previous_behavior
    return behavioral_model

# generate predictions on next state
def imc(system_estimate, previous_state, behavioral_model, previous_behavior):
    # actual estimate
    state = (system_estimate * previous_state) + (behavioral_model * previous_behavior)
    return state

# adjust system estimate to infer observations given my behavior
def imc_update(inputs, system_estimate, previous_behavior, behavioral_model, previous_state, learning_rate):
    # actual estimate
    curr_system_est = (system_estimate * previous_state) + (behavioral_model * previous_behavior)
    error = curr_system_est - inputs
    #learning_rate = 0.01
    system_estimate = system_estimate + learning_rate * error * previous_behavior
    return system_estimate
    
# behavior = last observation * behavioral model (like a motor command u)
# output = your behavior * est

In [ ]:
agent2 = Node()
agent1 = Node(sensor=sense, comparator=compare, control_update=control_update, controller = control, internal_model_update=imc_update, behavioral_model=B1, system_estimate=A1, reference=R1)
agent2 = Node(sensor=sense, comparator=compare, control_update=control_update,  controller = control, internal_model_update=imc_update, behavioral_model=B2, system_estimate=A2, reference=R2)


In [ ]:
trials = 100
behavior1 = np.array([0,0,0])
behavior2 = np.array([0,0,0])
agent1_behaviors = []
agent2_behaviors = []
agent1_error = []
agent2_error = []

for t in range(trials):
    b1 = agent1.go(behavior2)
    b2 = agent2.go(behavior1)

    agent1_behaviors.append(agent1.get_behavior())
    agent2_behaviors.append(agent2.get_behavior())
    agent1_behaviors.append(agent1.get_error())
    agent2_behaviors.append(agent2.get_error())

    behavior1 = b1
    behavior2 = b2
